In [1]:
pip install rasterio numpy


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install rasterio


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import os
import rasterio
from rasterio.merge import merge
from rasterio.enums import Resampling
from rasterio import Affine
from rasterio.shutil import copy as rio_copy

# Set your root directories
input_root = r"D:\F.work.DP\Data\sentinal 2A\raw data"
output_root = r"D:\F.work.DP\Data\sentinal 2A\process\mosaic data"

# Define years and bands
years = ["2016", "2018", "2020", "2022", "2024"]
bands = ["B2", "B3", "B4", "B8"]

for year in years:
    for band in bands:
        # Collect all .jp2 files for this band/year
        band_paths = []
        year_folder = os.path.join(input_root, year)
        
        # Walk through each tile folder under year
        for tile_folder in os.listdir(year_folder):
            tile_path = os.path.join(year_folder, tile_folder, f"{band}.jp2")
            if os.path.exists(tile_path):
                band_paths.append(tile_path)

        if len(band_paths) == 0:
            print(f"[!] No {band} files found for {year}. Skipping...")
            continue

        # Open and read all rasters
        src_files_to_mosaic = []
        for path in band_paths:
            src = rasterio.open(path)
            src_files_to_mosaic.append(src)

        # Perform the mosaic
        mosaic, out_transform = merge(src_files_to_mosaic, method='first')  # or method='mean'

        # Get metadata from first tile
        out_meta = src_files_to_mosaic[0].meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": mosaic.shape[1],
            "width": mosaic.shape[2],
            "transform": out_transform,
            "count": 1,
            "dtype": mosaic.dtype,
            "compress": "lzw"
        })

        # Create output directory for year
        out_dir = os.path.join(output_root, year)
        os.makedirs(out_dir, exist_ok=True)

        # Save the output mosaic
        out_path = os.path.join(out_dir, f"{band}_{year}_mosaic.tif")
        with rasterio.open(out_path, "w", **out_meta) as dest:
            dest.write(mosaic[0], 1)

        print(f"[✔] Mosaic saved: {out_path}")

# Done
print("✅ All mosaics completed.")


[✔] Mosaic saved: D:\F.work.DP\Data\sentinal 2A\process\mosaic data\2016\B2_2016_mosaic.tif
[✔] Mosaic saved: D:\F.work.DP\Data\sentinal 2A\process\mosaic data\2016\B3_2016_mosaic.tif
[✔] Mosaic saved: D:\F.work.DP\Data\sentinal 2A\process\mosaic data\2016\B4_2016_mosaic.tif
[✔] Mosaic saved: D:\F.work.DP\Data\sentinal 2A\process\mosaic data\2016\B8_2016_mosaic.tif
[✔] Mosaic saved: D:\F.work.DP\Data\sentinal 2A\process\mosaic data\2018\B2_2018_mosaic.tif
[✔] Mosaic saved: D:\F.work.DP\Data\sentinal 2A\process\mosaic data\2018\B3_2018_mosaic.tif
[✔] Mosaic saved: D:\F.work.DP\Data\sentinal 2A\process\mosaic data\2018\B4_2018_mosaic.tif
[✔] Mosaic saved: D:\F.work.DP\Data\sentinal 2A\process\mosaic data\2018\B8_2018_mosaic.tif
[✔] Mosaic saved: D:\F.work.DP\Data\sentinal 2A\process\mosaic data\2020\B2_2020_mosaic.tif
[✔] Mosaic saved: D:\F.work.DP\Data\sentinal 2A\process\mosaic data\2020\B3_2020_mosaic.tif
[✔] Mosaic saved: D:\F.work.DP\Data\sentinal 2A\process\mosaic data\2020\B4_2020

In [12]:
pip install rasterio geopandas shapely


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import os
import rasterio
from rasterio.mask import mask
import geopandas as gpd

# Set paths
input_root = r"D:\F.work.DP\Data\sentinal 2A\process\mosaic data"
output_root = r"D:\F.work.DP\Data\sentinal 2A\process\clip to diamer"
shapefile_path = r"D:\mtinan ata\gis data\Gb data\District Boundries\Diamer.shp"

# Read the shapefile
shape = gpd.read_file(shapefile_path)
shape = shape.to_crs("EPSG:32643")  # Assuming UTM Zone 43N (adjust if needed)
geom = [feature["geometry"] for feature in shape.__geo_interface__["features"]]

# Loop over years and clip each .tif
years = ["2016", "2018", "2020", "2022", "2024"]
bands = ["B2", "B3", "B4", "B8"]

for year in years:
    in_dir = os.path.join(input_root, year)
    out_dir = os.path.join(output_root, year)
    os.makedirs(out_dir, exist_ok=True)

    for band in bands:
        in_path = os.path.join(in_dir, f"{band}_{year}_mosaic.tif")
        out_path = os.path.join(out_dir, f"{band}_{year}_clipped.tif")

        if not os.path.exists(in_path):
            print(f"[!] Missing: {in_path}")
            continue

        with rasterio.open(in_path) as src:
            # Clip
            out_image, out_transform = mask(src, geom, crop=True)
            out_meta = src.meta.copy()
            out_meta.update({
                "driver": "GTiff",
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform,
                "compress": "lzw"
            })

            with rasterio.open(out_path, "w", **out_meta) as dest:
                dest.write(out_image)

        print(f"[✔] Clipped: {out_path}")

print("✅ All mosaics clipped to Diamer shapefile.")


[✔] Clipped: D:\F.work.DP\Data\sentinal 2A\process\clip to diamer\2016\B2_2016_clipped.tif
[✔] Clipped: D:\F.work.DP\Data\sentinal 2A\process\clip to diamer\2016\B3_2016_clipped.tif
[✔] Clipped: D:\F.work.DP\Data\sentinal 2A\process\clip to diamer\2016\B4_2016_clipped.tif
[✔] Clipped: D:\F.work.DP\Data\sentinal 2A\process\clip to diamer\2016\B8_2016_clipped.tif
[✔] Clipped: D:\F.work.DP\Data\sentinal 2A\process\clip to diamer\2018\B2_2018_clipped.tif
[✔] Clipped: D:\F.work.DP\Data\sentinal 2A\process\clip to diamer\2018\B3_2018_clipped.tif
[✔] Clipped: D:\F.work.DP\Data\sentinal 2A\process\clip to diamer\2018\B4_2018_clipped.tif
[✔] Clipped: D:\F.work.DP\Data\sentinal 2A\process\clip to diamer\2018\B8_2018_clipped.tif
[✔] Clipped: D:\F.work.DP\Data\sentinal 2A\process\clip to diamer\2020\B2_2020_clipped.tif
[✔] Clipped: D:\F.work.DP\Data\sentinal 2A\process\clip to diamer\2020\B3_2020_clipped.tif
[✔] Clipped: D:\F.work.DP\Data\sentinal 2A\process\clip to diamer\2020\B4_2020_clipped.tif

In [2]:
import os
import numpy as np
import rasterio

# Paths
input_root = r"D:\\F.work.DP\\Data\\sentinal 2A\\process\\clip to diamer"
output_root = r"D:\\F.work.DP\\Data\\sentinal 2A\\process\\Ndvi"
years = ["2016", "2018", "2020", "2022", "2024"]

for year in years:
    in_b4 = os.path.join(input_root, year, f"B4_{year}_clipped.tif")
    in_b8 = os.path.join(input_root, year, f"B8_{year}_clipped.tif")

    out_dir = os.path.join(output_root, year)
    os.makedirs(out_dir, exist_ok=True)
    out_ndvi = os.path.join(out_dir, f"NDVI_{year}.tif")

    if not (os.path.exists(in_b4) and os.path.exists(in_b8)):
        print(f"[!] Missing B4 or B8 for {year} — skipping.")
        continue

    with rasterio.open(in_b4) as red_src, rasterio.open(in_b8) as nir_src:
        red = red_src.read(1).astype("float32")
        nir = nir_src.read(1).astype("float32")

        # NDVI formula
        np.seterr(divide='ignore', invalid='ignore')
        ndvi = (nir - red) / (nir + red)
        ndvi = np.clip(ndvi, -1, 1)

        # Simple cloud/snow artifact filter
        # Clouds often have very low NIR and RED, or extreme NDVI
        invalid_mask = (nir + red) < 0.1  # very dark pixels
        ndvi[invalid_mask] = np.nan

        # Save NDVI raster
        meta = red_src.meta
        meta.update(driver="GTiff", dtype="float32", count=1, nodata=np.nan)

        with rasterio.open(out_ndvi, "w", **meta) as dst:
            dst.write(ndvi, 1)

    print(f"[✔] NDVI saved: {out_ndvi}")

print("✅ NDVI calculation completed for all years.")


[✔] NDVI saved: D:\\F.work.DP\\Data\\sentinal 2A\\process\\Ndvi\2016\NDVI_2016.tif
[✔] NDVI saved: D:\\F.work.DP\\Data\\sentinal 2A\\process\\Ndvi\2018\NDVI_2018.tif
[✔] NDVI saved: D:\\F.work.DP\\Data\\sentinal 2A\\process\\Ndvi\2020\NDVI_2020.tif
[✔] NDVI saved: D:\\F.work.DP\\Data\\sentinal 2A\\process\\Ndvi\2022\NDVI_2022.tif
[✔] NDVI saved: D:\\F.work.DP\\Data\\sentinal 2A\\process\\Ndvi\2024\NDVI_2024.tif
✅ NDVI calculation completed for all years.


In [3]:
import os
import rasterio
import numpy as np

# Paths
clip_path = r"D:\F.work.DP\Data\sentinal 2A\process\clip to diamer"
ndvi_path = r"D:\F.work.DP\Data\sentinal 2A\process\Ndvi"
out_path = r"D:\F.work.DP\Data\sentinal 2A\process\composite"
os.makedirs(out_path, exist_ok=True)

years = ["2016", "2018", "2020", "2022", "2024"]

for year in years:
    print(f"\n[📦] Processing {year}")

    # Input file list
    file_list = [
        os.path.join(clip_path, year, f"B2_{year}_clipped.tif"),
        os.path.join(clip_path, year, f"B3_{year}_clipped.tif"),
        os.path.join(clip_path, year, f"B4_{year}_clipped.tif"),
        os.path.join(clip_path, year, f"B8_{year}_clipped.tif"),
        os.path.join(ndvi_path, year, f"NDVI_{year}.tif")
    ]

    if not all(os.path.exists(f) for f in file_list):
        print(f"[!] Missing one or more bands for {year}, skipping.")
        continue

    arrays = []
    profile = None
    for i, path in enumerate(file_list):
        with rasterio.open(path) as src:
            if profile is None:
                profile = src.profile
            arrays.append(src.read(1).astype(np.float32))

    stacked = np.stack(arrays)  # shape: (5, H, W)
    profile.update(count=5, dtype="float32")

    out_file = os.path.join(out_path, f"Composite_{year}.tif")
    with rasterio.open(out_file, "w", **profile) as dst:
        dst.write(stacked)

    print(f"[✔] Composite saved: {out_file}")

print("\n✅ All composites created successfully.")



[📦] Processing 2016
[✔] Composite saved: D:\F.work.DP\Data\sentinal 2A\process\composite\Composite_2016.tif

[📦] Processing 2018
[✔] Composite saved: D:\F.work.DP\Data\sentinal 2A\process\composite\Composite_2018.tif

[📦] Processing 2020
[✔] Composite saved: D:\F.work.DP\Data\sentinal 2A\process\composite\Composite_2020.tif

[📦] Processing 2022
[✔] Composite saved: D:\F.work.DP\Data\sentinal 2A\process\composite\Composite_2022.tif

[📦] Processing 2024
[✔] Composite saved: D:\F.work.DP\Data\sentinal 2A\process\composite\Composite_2024.tif

✅ All composites created successfully.


In [4]:
import os
import rasterio
import numpy as np

def normalize_band(band):
    band = np.nan_to_num(band, nan=0.0)  # Replace NaN with 0
    min_val = np.nanmin(band)
    max_val = np.nanmax(band)
    if max_val - min_val == 0:
        return np.zeros_like(band, dtype=np.float32)
    return ((band - min_val) / (max_val - min_val)).astype(np.float32)

clip_path = r"D:\F.work.DP\Data\sentinal 2A\process\composite"
out_path = r"D:\F.work.DP\Data\sentinal 2A\process\composite_norm_allbands"
os.makedirs(out_path, exist_ok=True)

years = ["2016", "2018", "2020", "2022", "2024"]

for year in years:
    print(f"\n[📦] Normalizing all bands for {year}")
    comp_file = os.path.join(clip_path, f"Composite_{year}.tif")
    if not os.path.exists(comp_file):
        print(f"[!] Composite missing for {year}, skipping.")
        continue

    with rasterio.open(comp_file) as src:
        data = src.read().astype(np.float32)  # (bands, H, W)
        profile = src.profile.copy()

        # Normalize each band independently
        for b in range(data.shape[0]):
            data[b] = normalize_band(data[b])

        profile.update(dtype='float32')
        out_file = os.path.join(out_path, f"Composite_{year}_norm.tif")
        with rasterio.open(out_file, 'w', **profile) as dst:
            dst.write(data)

    print(f"[✔] Normalized composite saved: {out_file}")

print("\n✅ All composites normalized successfully.")



[📦] Normalizing all bands for 2016
[✔] Normalized composite saved: D:\F.work.DP\Data\sentinal 2A\process\composite_norm_allbands\Composite_2016_norm.tif

[📦] Normalizing all bands for 2018
[✔] Normalized composite saved: D:\F.work.DP\Data\sentinal 2A\process\composite_norm_allbands\Composite_2018_norm.tif

[📦] Normalizing all bands for 2020
[✔] Normalized composite saved: D:\F.work.DP\Data\sentinal 2A\process\composite_norm_allbands\Composite_2020_norm.tif

[📦] Normalizing all bands for 2022
[✔] Normalized composite saved: D:\F.work.DP\Data\sentinal 2A\process\composite_norm_allbands\Composite_2022_norm.tif

[📦] Normalizing all bands for 2024
[✔] Normalized composite saved: D:\F.work.DP\Data\sentinal 2A\process\composite_norm_allbands\Composite_2024_norm.tif

✅ All composites normalized successfully.


In [11]:
import os
import rasterio
import numpy as np
from rasterio.windows import Window

ndvi_path = r"D:\F.work.DP\Data\sentinal 2A\process\Ndvi"
mask_output_path = r"D:\F.work.DP\Data\sentinal 2A\process\forest_masks"
os.makedirs(mask_output_path, exist_ok=True)

years = ["2016", "2018", "2020", "2022", "2024"]
ndvi_threshold = 0.3

for year in years:
    ndvi_file = os.path.join(ndvi_path, year, f"NDVI_{year}.tif")
    if not os.path.exists(ndvi_file):
        print(f"[!] NDVI file missing for {year}, skipping.")
        continue

    with rasterio.open(ndvi_file) as src:
        profile = src.profile
        nodata = src.nodata

        # Update profile for mask output
        profile.update(
            dtype=rasterio.uint8,
            count=1,
            compress='lzw'
        )

        out_file = os.path.join(mask_output_path, f"ForestMask_{year}.tif")

        with rasterio.open(out_file, 'w', **profile) as dst:

            # Process in windows (blocks)
            for ji, window in src.block_windows(1):
                ndvi_block = src.read(1, window=window).astype(np.float32)

                if nodata is not None:
                    valid_mask = (ndvi_block != nodata)
                else:
                    valid_mask = np.ones_like(ndvi_block, dtype=bool)

                mask_block = ((ndvi_block > ndvi_threshold) & valid_mask).astype(np.uint8)

                dst.write(mask_block, 1, window=window)

    print(f"[✔] Forest mask saved for {year}: {out_file}")

print("\n✅ All forest masks created successfully.")


[✔] Forest mask saved for 2016: D:\F.work.DP\Data\sentinal 2A\process\forest_masks\ForestMask_2016.tif
[✔] Forest mask saved for 2018: D:\F.work.DP\Data\sentinal 2A\process\forest_masks\ForestMask_2018.tif
[✔] Forest mask saved for 2020: D:\F.work.DP\Data\sentinal 2A\process\forest_masks\ForestMask_2020.tif
[✔] Forest mask saved for 2022: D:\F.work.DP\Data\sentinal 2A\process\forest_masks\ForestMask_2022.tif
[✔] Forest mask saved for 2024: D:\F.work.DP\Data\sentinal 2A\process\forest_masks\ForestMask_2024.tif

✅ All forest masks created successfully.


In [ ]:
import os
import numpy as np
import rasterio

# Paths
composite_path = r"D:\F.work.DP\Data\sentinal 2A\process\composite"
mask_path = r"D:\F.work.DP\Data\sentinal 2A\process\forest_masks"
patch_output_path = r"D:\F.work.DP\Data\sentinal 2A\process\patches"
os.makedirs(patch_output_path, exist_ok=True)

patch_size = 256
stride = 256  # No overlap; change if you want overlapping patches
years = ["2016", "2018", "2020", "2022", "2024"]

def extract_patches_from_raster(raster_path, patch_size, stride):
    """Extract patches from a multi-band raster as numpy arrays."""
    patches = []
    with rasterio.open(raster_path) as src:
        bands = src.count
        height = src.height
        width = src.width

        # Slide over the image in strides
        for top in range(0, height - patch_size + 1, stride):
            for left in range(0, width - patch_size + 1, stride):
                window = rasterio.windows.Window(left, top, patch_size, patch_size)
                patch = src.read(window=window)  # shape: (bands, patch_size, patch_size)
                patches.append(patch.astype(np.float32))
    return np.array(patches)

def extract_patches_from_mask(mask_path, patch_size, stride):
    """Extract patches from single-band mask raster."""
    patches = []
    with rasterio.open(mask_path) as src:
        height = src.height
        width = src.width

        for top in range(0, height - patch_size + 1, stride):
            for left in range(0, width - patch_size + 1, stride):
                window = rasterio.windows.Window(left, top, patch_size, patch_size)
                patch = src.read(1, window=window)  # single band
                patches.append(patch.astype(np.uint8))
    return np.array(patches)

for year in years:
    composite_file = os.path.join(composite_path, f"Composite_{year}.tif")
    mask_file = os.path.join(mask_path, f"ForestMask_{year}.tif")

    if not os.path.exists(composite_file) or not os.path.exists(mask_file):
        print(f"[!] Missing composite or mask for {year}, skipping.")
        continue

    print(f"[📦] Extracting patches for {year}...")

    img_patches = extract_patches_from_raster(composite_file, patch_size, stride)
    mask_patches = extract_patches_from_mask(mask_file, patch_size, stride)

    # Sanity check
    assert img_patches.shape[0] == mask_patches.shape[0], "Mismatch in patch count!"

    np.save(os.path.join(patch_output_path, f"{year}_images.npy"), img_patches)
    np.save(os.path.join(patch_output_path, f"{year}_masks.npy"), mask_patches)

    print(f"[✔] Saved {img_patches.shape[0]} patches for {year}")

print("\n✅ All patches extracted and saved successfully.")


[📦] Extracting patches for 2016...
[✔] Saved 2106 patches for 2016
[📦] Extracting patches for 2018...
[✔] Saved 2106 patches for 2018
[📦] Extracting patches for 2020...
[✔] Saved 2106 patches for 2020
[📦] Extracting patches for 2022...
[✔] Saved 2106 patches for 2022
[📦] Extracting patches for 2024...
